# Comparación de métodos

In [2]:
#Importamos las modulos
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [3]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [4]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [5]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [6]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [7]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [8]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosFB = []
puntosSD = []
puntosN = []
puntosM = []
puntosGC = []
puntosBFP = []
puntosBFGS = []
times = []
iters = []
pmins = []
xoptm = []

In [9]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1.3, 1.3, 100)
    y = np.linspace(-3, 1.3, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

## Brute Force

In [10]:
start_time = time()
xi_fb = [-1,1] #Punto de partida
xa_fb =  xi_fb
alpha = 0.001
iteraciones = 1
fx_value = f(xi_fb) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_value)
for i in range(1000): #Proceso iterativo del algoritmo
    xa_fb =  xi_fb
    si = gradient(xi_fb, delta)
    s1 = -1*si[0]
    s2 = -1*si[1]
    xdd = xi_fb[0] + alpha*s1
    ydd = xi_fb[1] + alpha*s2
    fdd1 = f([xdd, ydd])
    fdd2 = f(xi_fb)
    if(abs(fdd1-fdd2)<=ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi_fb[0], xi_fb[1], f(xi_fb), LA.norm(si)))
        break
    xi_fb = [xdd, ydd]
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi_fb[0], xi_fb[1], f(xi_fb), LA.norm(si)))
    puntosFB.append([[xa_fb[0],xi_fb[0]],[xa_fb[1],xi_fb[1]]]) #Almacenamos los punto de direccion del algoritmo
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosFB, 'Brute Force')
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(xi_fb))
xoptm.append(xi_fb)

Valor de la función inicial = 270.294169 
0	[-0.7699,0.6199]		110.2512		444.3158
1	[-0.6205,0.3875]		48.2750		276.2647
2	[-0.5198,0.2472]		23.9401		172.7293
3	[-0.4485,0.1640]		14.0401		109.5750
4	[-0.3947,0.1163]		9.6750		71.9210
5	[-0.3514,0.0903]		7.4400		50.4692
6	[-0.3144,0.0777]		6.0375		39.0719
7	[-0.2814,0.0730]		4.9762		33.3881
8	[-0.2508,0.0728]		4.0712		30.5303
9	[-0.2220,0.0752]		3.2533		28.8984
10	[-0.1945,0.0787]		2.4967		27.7526
11	[-0.1680,0.0828]		1.7909		26.7903
12	[-0.1425,0.0868]		1.1314		25.8974
13	[-0.1177,0.0906]		0.5152		25.0343
14	[-0.0937,0.0940]		-0.0599		24.1892
15	[-0.0706,0.0969]		-0.5961		23.3599
16	[-0.0482,0.0993]		-1.0955		22.5472
17	[-0.0265,0.1012]		-1.5602		21.7524
18	[-0.0056,0.1026]		-1.9923		20.9767
19	[0.0146,0.1036]		-2.3938		20.2211
20	[0.0341,0.1042]		-2.7665		19.4860
21	[0.0529,0.1044]		-3.1124		18.7720
22	[0.0710,0.1043]		-3.4332		18.0792
23	[0.0884,0.1039]		-3.7305		17.4074
24	[0.1051,0.1032]		-4.0060		16.7568
25	[0.1212,0.1023]		-4.2612		

## Steepest descent method

In [11]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
xi_sd = [-1,1] #Punto de partida
xi_1 = [-1,1] #Punto inicial para el metodo del golden section
fx_value = f(xi_sd) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_value)
for i in range(1000): #Proceso iterativo del algoritmo
    grad = gradient(xi_1,delta) #Se obtiene el vector del gradiente de la funcion objetivo
    si = -grad #La direccion de busqueda es el gradiente negativo
    alpha, fx_curr = golden(xi_1,si,xi_sd,ep1) #Se encuentra el alpha optimo (tamaño de paso)
    x_opt = xi_1 + alpha*si #Se encuentra el nuevo optimo
    if abs(fx_curr-fx_value)<ep1 or LA.norm(grad)<ep1: #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x_opt[0], x_opt[1],fx_value,LA.norm(grad)))
        break
    fx_value=fx_curr #Ahora fx_value se iguala con el actual, de esta forma el actual ahora es el anterior
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x_opt[0], x_opt[1],fx_value,LA.norm(grad)))
    puntosSD.append([[xi_1[0],x_opt[0]],[xi_1[1],x_opt[1]]]) #Almacenamos los punto de direccion del algoritmo
    xi_1=x_opt #ahora x anterior es x_opt
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosSD, 'Steepest descent method')
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(x_opt))
xoptm.append(x_opt)

Valor de la función inicial = 270.294169 
0	[-0.3892,-0.0092]		8.9425		444.3158
1	[-0.0345,0.2013]		0.3751		33.6772
2	[0.0410,0.0766]		-2.7472		42.8521
3	[0.1616,0.1444]		-4.3989		21.6597
4	[0.2127,0.0633]		-5.4337		21.9813
5	[0.2783,0.1043]		-6.0764		16.1606
6	[0.3148,0.0471]		-6.5224		13.2170
7	[0.3540,0.0712]		-6.8064		11.4880
8	[0.3868,0.0277]		-7.0162		7.8351
9	[0.4105,0.0454]		-7.1464		8.5604
10	[0.4349,0.0157]		-7.2356		4.5741
11	[0.4483,0.0261]		-7.2851		5.4709
12	[0.4691,0.0061]		-7.3234		2.4656
13	[0.4763,0.0126]		-7.3382		3.1097
14	[0.4833,0.0040]		-7.3451		1.1885
15	[0.4869,0.0067]		-7.3485		1.4562
16	[0.4926,0.0006]		-7.3509		0.6321
17	[0.4930,0.0011]		-7.3509		0.9605
Tiempo transcurrido: 8.971453
Iteraciones: 18


## Modified Newton’s method

In [12]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
xi = [-1,1] #Punto de partida
x = xi #Punto inicial para el metodo de golden section
xAnt = xi #Definimos una variable de punto anterior
fx_prev = f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_prev)
for i in range(1000): #Proceso iterativo del algoritmo
    xAnt = x #Almacenamos el punto anterior
    dire = gradient(x,delta) #Se obtiene la matriz del gradiente de la funcion objetivo
    H = Hessian(x,delta) #Se calcula la matriz Hessiana
    dire = np.atleast_2d(dire) #La función se usa cuando queremos convertir entradas en matrices con al menos dos dimensiones. Las entradas escalares y unidimensionales se convierten en matrices bidimensionales, mientras que las entradas de mayor dimensión se conservan.
    si=np.matmul(-LA.inv(H),dire.transpose()) #Se multiplica la inversa de la matriz Hessina por el vector gradiente (de la direccion)
    si = si.transpose() #Ahora la nueva direccion de busqueda es la transpuesta esto porque la multiplicacion
    si = np.ndarray.flatten(si) #Devuelve una copia de la matriz colapsada en una dimensión.
    alpha, fx_curr = golden(x,si,xi,ep1) #Se encuentra el alpha optimo (tamaño de paso)
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire)<ep1: #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
        break;
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
    fx_prev=fx_curr
    x = x +  alpha*si #ahora x anterior es x_opt
    puntosN.append([[xAnt[0],x[0]],[xAnt[1],x[1]]])
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosN, 'Modified Newton’s method') 
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(x))
xoptm.append(x)

Valor de la función inicial = 270.294169 
0	[-1.0000,1.0000]		6.5243		444.3158
1	[-0.3266,0.0340]		-3.2287		31.1194
2	[0.1506,0.1848]		-6.8860		34.9350
3	[0.3772,0.0722]		-7.3383		9.3801
4	[0.4844,0.0138]		-7.3515		2.3801
5	[0.4948,0.0042]		-7.3521		0.7266
Tiempo transcurrido: 3.987074
Iteraciones: 6


## Marquardt’s method

In [13]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
I = np.identity(2) #Se establece la matriz identidad 2x2
labda = 10000 #Se define un labda inicial
x = [-1,1] #Punto de partida
x_prev = x #Definimos una variable de punto anterior
fx_prev=f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_prev)
for i in range(1000): #Proceso iterativo del algoritmo
    xAntLM = x #Almacenamos el punto anterior
    G = gradient(x,delta) #Se calcula el vector gradiente
    H = Hessian(x,delta) #Se calcula la matriz Hessiana
    S = -np.matmul(np.linalg.inv((H+labda*I)),G) #Se realiza la multiplicacion del resultado la inversa Hessiana summada la matriz Identidad multiplicada por labda, el resultado es multiplicado por el gradiente de la funcion
    x=x+S #Se actuliza el nuevo punto optimo
    fx=f(x) #Se evalua la funcion con el nuevo punto optimo
    if(fx<fx_prev): #Si fx es menor al previo la nueva labda sera multiplicada por 0.5 y se tendra un nuevo tamaño de paso
        labda=.5*labda
    else: #Caso contrario labda sera multiplicado por 2
        labda=2*labda
    #Almacenamos los puntos de direccion para trazar la grafoica
    puntosM.append([[x[0],xAntLM[0]],[x[1],xAntLM[1]]])  
    #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
    if(abs(fx_prev - fx) < ep1 or LA.norm(G)<ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_prev,LA.norm(G)))
        break
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_prev,LA.norm(G)))
    fx_prev=fx #Ahora fx sera fx anterior     
    iteraciones+=1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosM, 'Marquardt’s method') 
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(x))
xoptm.append(x)

Valor de la función inicial = 270.294169 
0	[-0.9778,0.9634]		270.2942		444.3158
1	[-0.9363,0.8955]		251.6203		428.0979
2	[-0.8635,0.7779]		218.7471		397.9601
3	[-0.7492,0.5990]		167.3425		345.6663
4	[-0.5978,0.3806]		102.4403		265.5825
5	[-0.4329,0.1956]		45.0861		166.5017
6	[-0.2585,0.1152]		14.8448		79.3220
7	[-0.0282,0.1225]		4.5436		35.1505
8	[0.2325,0.1119]		-1.4682		22.1993
9	[0.3997,0.0554]		-5.6152		14.1528
10	[0.4757,0.0147]		-7.0701		6.2961
11	[0.4968,0.0015]		-7.3353		1.6001
12	[0.4997,-0.0002]		-7.3527		0.1763
Tiempo transcurrido: 2.990007
Iteraciones: 13


## Conjugate gradient method

In [14]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
x = [-1,1] #Punto de partida
x_prev1 = x #Definimos una variable de punto anterior
x_prev = x #Definimos una variable de punto anterior
f_prev = f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % f_prev)
dire_prev=gradient(x,delta)
si_prev = -dire_prev
#x,search,xi,eps
alpha, fx_prev = golden(x,si_prev,x_prev,ep1);
x = x +  alpha*si_prev 
for i in range(1000): #Proceso iterativo del algoritmo
    dire=gradient(x,delta)
    si = - dire +((LA.norm(dire)**2)/(LA.norm(dire_prev)**2))*si_prev;
    alpha, fx_curr = golden(x,si,x_prev,ep1);
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire)<ep1:
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
        break;
    dire_prev=dire
    si_prev=si
    x = x +  (alpha*si).transpose() 
    fx_prev=f(x)
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
    puntosGC.append([[x_prev1[0],x[0]],[x_prev1[1],x[1]]])
    x_prev1 = x
    iteraciones+=1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosGC, 'Conjugate gradient method')  
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(x))
xoptm.append(x)

Valor de la función inicial = 270.294169 
0	[0.030,0.199]	-1.026	33.677
1	[0.438,0.101]	-6.301	39.919
2	[0.502,-0.001]	-7.353	26.034
3	[0.502,-0.001]	-7.353	0.224
Tiempo transcurrido: 2.990007
Iteraciones: 4


## DFP method

In [15]:
start_time = time()
xi = [-1,1]
x = xi
Bi = np.eye(len(x))
fx_prev = f(x)
print('Valor de la función inicial = %f ' % fx_prev) #Valor inicial de la funcion con el punto inicial
iteraciones = 1 #Conteo de numero de iteraciones
Gx = gradient(x,delta) #Direccion de busqueda
S = -Gx 
alpha,fx_prev = golden(x,S,xi,ep1)
xi_1 = x + alpha*S       
for j in range(1000):
    deltax=xi_1-x
    Gxi=gradient(xi_1,delta)
    Gi = Gxi-Gx
    Bi = Bi + (np.matmul(np.reshape(deltax,(2,1)),np.reshape(deltax,(1,2)))/np.matmul(deltax,Gi.transpose())) - (np.matmul(np.matmul(np.matmul(Bi, np.reshape(Gi,(2,1))), np.reshape(Gi,(1,2))),Bi) / np.matmul(np.matmul(np.reshape(Gi,(1,2)),Bi), np.reshape(Gi,(2,1))))
    si = np.matmul(-Bi,Gxi.transpose())
    si = np.ndarray.flatten(si.transpose())
    alpha,fx_curr = golden(xi_1[:],si,xi,ep1)
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(j, x[0], x[1],fx_curr,LA.norm(Gx)))
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(Gx)<ep1:
        break
    fx_prev=fx_curr
    Gx = Gxi
    iteraciones+=1
    puntosBFP.append([[x[0],xi_1[0]],[x[1],xi_1[1]]])
    x=xi_1
    xi_1 = x + alpha*si
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
graficaUnaaUna(puntosBFP, 'BFP')   
times.append(elapsed_time)
iters.append(j)
pmins.append(f(xi_1))
xoptm.append(xi_1)

Valor de la función inicial = 270.294169 
0	[-1.000,1.000]	-0.864	444.316
1	[-0.389,-0.009]	-4.201	33.677
2	[0.022,0.199]	-6.867	40.221
3	[0.200,0.168]	-7.267	30.562
4	[0.364,0.068]	-7.349	7.573
5	[0.475,0.031]	-7.352	6.891
6	[0.490,0.007]	-7.352	0.861
Tiempo transcurrido: 3.985643
Iteraciones: 7


## BFGS method

In [16]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
#Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity,
#take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
#1. Start with an initial point X1
xi = [-1,1]
xi_1 = xi
#n × n positive definite symmetric matrix [B1] as an initial estimate of the inverse of the Hessian matrix of f
#In the absence of additional information, [B1] is taken as the identity matrix [I].
Bi = np.eye(len(xi))
print(f(xi))
fx_prev=f(xi)
gF_xi = gradient(xi_1, delta)
#2. Compute the gradient of the function, ∇fi, at point Xi, and set
Si = np.matmul(Bi,gF_xi)
#3. Find the optimal step length λ∗i in the direction Si and set
alpha, fx = golden(xi_1,Si,xi,ep1)
xi_1 = xi - alpha*Si
for i in range(1000): #Proceso iterativo del algoritmo
    #Compute the gradient vector ∇f1 = ∇f(X1) and set the iteration number as i = 1.
    gF_xi = gradient(xi, delta)
    #2. Compute the gradient of the function, ∇fi, at point Xi, and set
    Si = np.matmul(np.dot(1, Bi),gF_xi)
    #3. Find the optimal step length λ∗i in the direction Si and set
    alpha, fx = golden(xi_1,Si,xi,ep1)
    xi_1 = xi + np.dot(alpha, Si)
    #4. Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity, take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
    if(LA.norm(gF_xi)<ep1 or abs(fx-fx_prev)<ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi[0], xi[1],fx,LA.norm(gF_xi)))
        break
    else:
        g1 = np.reshape(gradient(xi_1, delta),(2,1)) - np.reshape(gF_xi,(2,1))
        d1 = np.reshape(np.array(xi),(2,1)) - np.reshape(xi_1,(2,1))
        d1d1t = np.dot(d1, np.reshape(d1,(1,2)))
        d1tg1 = np.matmul(d1.transpose(),g1)
        d1g1t = np.matmul(d1, g1.transpose())
        g1d1t = np.matmul(g1, d1.transpose())
        g1tb1g1 = np.matmul(np.matmul(g1.transpose(), Bi), g1)
        d1g1tb1 = np.matmul(d1g1t,Bi)
        b1g1d1t = np.matmul(Bi, g1d1t)
        Bi = Bi + ((1+(g1tb1g1/d1tg1))*(d1d1t/d1tg1)) - (d1g1tb1/d1tg1) - (b1g1d1t/d1tg1) 
    puntosBFGS.append([[xi_1[0],xi[0]],[xi_1[1],xi[1]]])
    xi = xi_1
    fx_prev = fx
    iteraciones += 1
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi[0], xi[1],fx,LA.norm(gF_xi)))
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosBFGS, 'BFGS method')    
times.append(elapsed_time)
iters.append(iteraciones)
pmins.append(f(xi))
xoptm.append(xi)

270.29416855008
0	[1.2420,-2.7043]		55.8889		444.3158
1	[0.1960,-1.9523]		74.3165		443.6934
2	[1.1232,-1.2909]		34.0478		37.0109
3	[1.2709,-0.9555]		26.8304		67.5989
4	[0.9382,-0.1791]		-0.0429		90.7990
5	[0.8071,-0.2531]		-4.1299		55.0008
6	[0.6996,-0.1247]		-6.2440		16.2680
7	[0.5770,-0.0139]		-7.1258		9.5776
8	[0.5498,-0.0090]		-7.2601		9.9599
9	[0.5271,-0.0059]		-7.3280		6.2747
10	[0.5150,-0.0034]		-7.3452		3.0428
11	[0.5094,-0.0023]		-7.3499		1.6811
12	[0.5058,-0.0015]		-7.3518		1.0342
13	[0.5058,-0.0015]		-7.3525		0.6400
Tiempo transcurrido: 6.976604
Iteraciones: 14


## Comparación de métodos

In [17]:
def comparacionMetodos(puntosMetodos, nombresMetodos):
    x = np.linspace(-1, 1.3, 100)
    y = np.linspace(-3, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x, y, z, 20)
    plt.plot(-1, 1, 'ro--',  markersize=6)
    plt.plot(0.5, 0, 'go--', markersize=6)
    i = 0    
    for metodo in puntosMetodos:
        #Generamos valores ramdom para generar un color diferente de grafica
        r = random.random()
        b = random.random()
        g = random.random()
        color = (r, g, b)
        aux = 0
        #Graficamos los trazos (direcciones) que va generando el algoritmo
        for pts in metodo:
            if(aux==len(metodo)-1):
                plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombresMetodos[i]))
            else:
                plt.plot(pts[0],pts[1], c=color, linewidth=2)
            aux = aux + 1
        i += 1
    plt.legend()    
    plt.show

In [18]:
nombres = ['Force Brute','Steepest descent method','Modified Newton’s method','Marquardt’s method','Conjugate gradient method', 'BFP method','BFGS method']
listaPuntos = [puntosFB, puntosSD, puntosN, puntosM, puntosGC, puntosBFP, puntosBFGS]
comparacionMetodos(listaPuntos, nombres)

In [19]:
def estadisticas(metrica, nombres, ctrol):
    y = metrica
    z = np.arange(0,len(metrica),1)
    n = nombres
    fig, ax = plt.subplots()
    ax.scatter(z, y)
    ax.annotate(n[0], (z[0], y[0]), xytext=(z[0]+0.4, y[0]+0.3), 
        arrowprops=dict(facecolor='red', shrink=0.05))
    ax.annotate(n[1], (z[1], y[1]), xytext=(z[1]-0.4, y[1]-0.3), 
        arrowprops = dict(  arrowstyle="->",
                            connectionstyle="angle3,angleA=0,angleB=-90"))
    ax.annotate(n[2], (z[2], y[2]), xytext=(z[2]-0.4, y[2]-0.3), 
        arrowprops = dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1))
    ax.annotate(n[3], (z[3], y[3]), xytext=(z[3]+0.4, y[3]-0.2), 
        arrowprops = dict(arrowstyle="fancy"))
    ax.annotate(n[4], (z[4], y[4]), xytext=(z[4]-0.1, y[4]-0.2),
        bbox=dict(boxstyle="round", alpha=0.1), 
        arrowprops = dict(arrowstyle="simple"))
    ax.annotate(n[5], (z[5], y[5]), xytext=(z[5]+0.4, y[5]-0.2), 
        arrowprops = dict(arrowstyle="fancy"))
    if(ctrol==1):
        plt.xlabel("Método")
        plt.ylabel("Tiempo (s)")
    else:
        plt.xlabel("Método")
        plt.ylabel("Iteraciones")
    plt.show()

In [20]:
estadisticas(times, nombres, 1)
estadisticas(iters, nombres, 0)
print(pmins)

[-7.3406212644024365, -7.351450710528228, -7.3514921847513435, -7.352937586742434, -7.352742484122075, -7.352054497609378, -7.351785791967479]
